Crawler para extração dos itens do carrinho da Atividade Prática.

# Imports

In [19]:
import requests, json, threading, sys, traceback
import pandas as pd

# Funções

In [20]:
def get_category(category: str) -> list:
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    payload = {
        "partner": "linx",
        "page": 1,
        "resultsPerPage": 300,
        "multiCategory": category,
        "sortBy": "relevance",
        "department": "ecom",
        "storeId": 461,
        "customerPlus": True
    }
    
    try:
        # Pega o número total de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        max_produtos = response.json()['totalProducts']
        payload['resultsPerPage'] = max_produtos

        # Pega a lista de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        
        lista_p = list(response.json()['products'])
        lista_cod = []

        for prod in lista_p:
            lista_cod.append(prod['id'])
            
    except:
        return []
    
    else:
        return lista_cod

In [21]:
def get_product(code: int, verbose: int = 0) -> dict:
    try:
        # Fazendo a requisição
        texto = requests.get(fr"https://www.paodeacucar.com/produto/{code}").text
        
        # Encontrando as posições
        inicio = texto.find('"props":') + len('"props":')
        fim = texto.find(',"page":"/produto"', inicio)

        # Extraindo a substring
        substring = texto[inicio:fim].strip()
        
        # Dicionario Geral
        dicionario = json.loads(substring)
        
        # Informações gerais do produto
        informacoes = dicionario['initialProps']['componentProps']['product']
        atributos = get_dictkey(informacoes, 'attributeGroups')
        
        # Dicionário com os resultados
        resultados = {    
        'id': informacoes['id'], 
        'nome': informacoes['name'], 
        'sku': informacoes['sku'], 
        'descricao': informacoes['description'],
        'imagem': f'https://static.paodeacucar.com{informacoes['thumbPath']}',
        'preco': get_dictkey(informacoes, 'currentPrice'),
        'marca': get_dictcode(atributos, 'Marca'),
        'alturaProduto': get_dictcode(atributos, 'alturaProduto'),
        'larguraProduto': get_dictcode(atributos, 'larguraProduto'),
        'pesoBruto': get_dictcode(atributos, 'pesoBruto'),
        'unidadePesoBruto': get_dictcode(atributos, 'unidadePesoBruto'),
        'profundidadeProduto': get_dictcode(atributos, 'profundidadeProduto')
        }
        
        assert(
            resultados['id'] == code and 
            resultados['preco'] != None and
            resultados['alturaProduto'] != None and 
            resultados['larguraProduto'] != None and 
            resultados['pesoBruto'] != None and 
            resultados['unidadePesoBruto'] != None and
            resultados['profundidadeProduto'] != None and
            'kit' not in resultados['nome']
        )

    except:
        if verbose >= 1:
            exc_info = sys.exc_info()
            e = ''.join(traceback.format_exception(*exc_info))
            del exc_info
            
            print(f"Erro ao extrair informações do produto {code}!\nErro:\n{e}")
        
        if verbose >= 2:     
            if 'informacoes' in locals():
                print(json.dumps(locals()['informacoes'], indent=2))
                
            if 'informacoes2' in locals():
                print(json.dumps(locals()['informacoes2'], indent=2))
        
        return None
    
    else:
        return resultados

In [22]:
def fetch_products(codigos, verbose: int = 0):
    resultados = []
    threads = []
    lock = threading.Lock()

    def thread_function(codigo, verbose):
        info = get_product(codigo, verbose)
        
        if info:
            with lock:
                resultados.append(info)

    for codigo in codigos:
        thread = threading.Thread(target=thread_function, args=(codigo, verbose,))
        threads.append(thread)
        thread.start()

    while any(thread.is_alive() for thread in threads): pass

    return resultados


In [23]:
def get_dictkey(dicionario, chave):
    # Função interna recursiva
    def busca_recursiva(d):
        if isinstance(d, dict):
            for k, v in d.items():
                if k == chave:
                    return v
                resultado = busca_recursiva(v)
                if resultado is not None:
                    return resultado
                
        elif isinstance(d, list):
            for item in d:
                resultado = busca_recursiva(item)
                if resultado is not None:
                    return resultado
        return None

    return busca_recursiva(dicionario)

In [24]:
def get_dictcode(d, code):
    # Se d for um dicionário, iteramos sobre suas chaves e valores
    if isinstance(d, dict):
        if d.get('code') == code:
            return d.get('value')  # Retorna o valor se a chave 'code' for encontrada com o valor especificado
        for value in d.values():
            result = get_dictcode(value, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    # Se d for uma lista, iteramos sobre os elementos da lista
    elif isinstance(d, list):
        for item in d:
            result = get_dictcode(item, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    return None  # Retorna None se o dicionário não for encontrado

In [25]:
def format_desc(x):
    for rep in ['<b>', '</b>', '<br>', '</br>', '<p>', '</p>', '\r', '\n', r'"""¿']:
        x = x.replace(rep, '')
    
    return x

# Constants

In [26]:
CATEGORIAS = [
    'alimentos', 
    'beleza-e-perfumaria',
    'bebidas', 
    'bebidas-alcoolicas-', 
    'limpeza', 
    # 'bebes-e-criancas', 
    'cuidados-pessoais', 
    # 'suplementos-alimentares', 
    # 'eventos-e-festas',
    # 'utensilios-e-descartaveis',
    # 'petshop', 
    # 'esporte-e-lazer',
    # 'casa-e-construcao',
    # 'floricultura-e-jardim'
]

# Code

In [27]:
map = []
from crawler2 import *
dicio = main()

for category in list(dicio.keys()):
    print(f"Obtendo códigos de produtos da categoria {category}...")
    codigos = dicio[category]
    
    print(f"Obtendo informações dos produtos da categoria {category}...")
    # map[category] = fetch_products(codigos)
    for item in fetch_products(codigos, 0):
        map.append({**item, **{'categoria': category}})

map

Obtendo códigos de produtos da categoria alimentos...
Obtendo informações dos produtos da categoria alimentos...
Obtendo códigos de produtos da categoria beleza-e-perfumaria...
Obtendo informações dos produtos da categoria beleza-e-perfumaria...
Obtendo códigos de produtos da categoria bebidas...
Obtendo informações dos produtos da categoria bebidas...
Obtendo códigos de produtos da categoria bebidas-alcoolicas...
Obtendo informações dos produtos da categoria bebidas-alcoolicas...
Obtendo códigos de produtos da categoria limpeza...
Obtendo informações dos produtos da categoria limpeza...
Obtendo códigos de produtos da categoria cuidados-pessoais...
Obtendo informações dos produtos da categoria cuidados-pessoais...


[{'id': 116925,
  'nome': 'Sobrecoxa de Frango Congelada Sadia 1kg',
  'sku': '1171686',
  'descricao': '<b>Ingredientes:</b><br> Sobrecoxa de frango.',
  'imagem': 'https://static.paodeacucar.com/img/uploads/1/388/26941388.jpg',
  'preco': 14.9,
  'marca': 'Sadia',
  'alturaProduto': '23.6',
  'larguraProduto': '18.2',
  'pesoBruto': '1.06',
  'unidadePesoBruto': 'KG',
  'profundidadeProduto': '3.2',
  'categoria': 'alimentos'},
 {'id': 16778,
  'nome': 'Rabanete Bandeja 400g',
  'sku': '4337409',
  'descricao': '<b>Ingrediente:</b><br>Rabanete.<br><b>NÃO CONTÉM GLÚTEN.</b>',
  'imagem': 'https://static.paodeacucar.com/img/uploads/1/60/601060.JPG',
  'preco': 7.19,
  'marca': 'Bandeiras',
  'alturaProduto': '6.1',
  'larguraProduto': '14.5',
  'pesoBruto': '416',
  'unidadePesoBruto': 'G',
  'profundidadeProduto': '15.37',
  'categoria': 'alimentos'},
 {'id': 135368,
  'nome': 'Alho Porró Orgânico TAEQ 250g',
  'sku': '3466537',
  'descricao': '<b>Ingredientes:</b><br>Alho porró. <br>

In [28]:
df = pd.concat([pd.DataFrame([item]) for item in map], ignore_index=True, sort=False, axis='index')
df

,id,nome,sku,descricao,imagem,preco,marca,alturaProduto,larguraProduto,pesoBruto,unidadePesoBruto,profundidadeProduto,categoria
0,116925,Sobrecoxa de Frango Congelada Sadia 1kg,1171686,<b>Ingredientes:</b><br> Sobrecoxa de frango.,https://static.paodeacucar.com/img/uploads/1/3...,14.90,Sadia,23.6,18.2,1.06,KG,3.2,alimentos
1,16778,Rabanete Bandeja 400g,4337409,<b>Ingrediente:</b><br>Rabanete.<br><b>NÃO CON...,https://static.paodeacucar.com/img/uploads/1/6...,7.19,Bandeiras,6.1,14.5,416,G,15.37,alimentos
2,135368,Alho Porró Orgânico TAEQ 250g,3466537,<b>Ingredientes:</b><br>Alho porró. <br><b>NÃO...,https://static.paodeacucar.com/img/uploads/1/8...,9.19,Taeq,4.5,25,360,G,30,alimentos
3,262011,Suco de Uva Orgânico 100% Bordô UVA'SÓ Garrafa...,1010560,SUCO UVA ORG BORDO 300ML.,https://static.paodeacucar.com/img/uploads/1/5...,18.90,Bandeiras,21,6,560,G,6,alimentos
4,158920,Repolho Verde Orgânico TAEQ 400g,3467442,<b>Ingrediente:</b><br>Repolho. <br><b>NÃO CON...,https://static.paodeacucar.com/img/uploads/1/2...,8.99,Taeq,5,15,400,G,15,alimentos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15250,320688,Enxaguante Bucal Ice Infinity Rock in Rio Colg...,1070182,O Enxaguante Bucal Colgate Plax Ice Infinity p...,https://static.paodeacucar.com/img/uploads/1/7...,21.99,Colgate,22.2,9.7,564,G,5,cuidados-pessoais
15251,545529,Protetor Solar Aerosol Above FPS50,1361062,O Protetor Solar Aerosol Above FPS 50 ajuda a ...,https://static.paodeacucar.com/img/uploads/1/4...,57.90,Above,18.4,4.5,161,G,4.5,cuidados-pessoais
15252,386314,Enxaguante Bucal Antisséptico Zero Álcool Ment...,1127672,"""O que é e para que serve? \r\nVocê sabia que ...",https://static.paodeacucar.com/img/uploads/1/9...,35.99,Listerine,22,9,600,G,5,cuidados-pessoais
15253,591435,Australian Gold Protetor Solar Corporal FPS 50...,1311546,Australian Gold Protetor Solar Corporal FPS 50...,https://static.paodeacucar.com/img/uploads/1/5...,75.90,Australian Gold,20,4.6,227,G,7.8,cuidados-pessoais


In [29]:
df['descricao'] = df['descricao'].apply(format_desc)
df['descricao'] = df['descricao'].fillna('Sem descrição')
df['pesoBruto'] = df['pesoBruto'].astype(float)
df['pesoBruto'] = df.apply(lambda x: x['pesoBruto'] if x['unidadePesoBruto'] == 'G' else x['pesoBruto'] * 1000, axis=1)
df.drop(columns=['unidadePesoBruto'], inplace=True)
df

,id,nome,sku,descricao,imagem,preco,marca,alturaProduto,larguraProduto,pesoBruto,profundidadeProduto,categoria
0,116925,Sobrecoxa de Frango Congelada Sadia 1kg,1171686,Ingredientes: Sobrecoxa de frango.,https://static.paodeacucar.com/img/uploads/1/3...,14.90,Sadia,23.6,18.2,1060.0,3.2,alimentos
1,16778,Rabanete Bandeja 400g,4337409,Ingrediente:Rabanete.NÃO CONTÉM GLÚTEN.,https://static.paodeacucar.com/img/uploads/1/6...,7.19,Bandeiras,6.1,14.5,416.0,15.37,alimentos
2,135368,Alho Porró Orgânico TAEQ 250g,3466537,Ingredientes:Alho porró. NÃO CONTÉM GLÚTEN.,https://static.paodeacucar.com/img/uploads/1/8...,9.19,Taeq,4.5,25,360.0,30,alimentos
3,262011,Suco de Uva Orgânico 100% Bordô UVA'SÓ Garrafa...,1010560,SUCO UVA ORG BORDO 300ML.,https://static.paodeacucar.com/img/uploads/1/5...,18.90,Bandeiras,21,6,560.0,6,alimentos
4,158920,Repolho Verde Orgânico TAEQ 400g,3467442,Ingrediente:Repolho. NÃO CONTÉM GLÚTEN.,https://static.paodeacucar.com/img/uploads/1/2...,8.99,Taeq,5,15,400.0,15,alimentos
...,...,...,...,...,...,...,...,...,...,...,...,...
15250,320688,Enxaguante Bucal Ice Infinity Rock in Rio Colg...,1070182,O Enxaguante Bucal Colgate Plax Ice Infinity p...,https://static.paodeacucar.com/img/uploads/1/7...,21.99,Colgate,22.2,9.7,564.0,5,cuidados-pessoais
15251,545529,Protetor Solar Aerosol Above FPS50,1361062,O Protetor Solar Aerosol Above FPS 50 ajuda a ...,https://static.paodeacucar.com/img/uploads/1/4...,57.90,Above,18.4,4.5,161.0,4.5,cuidados-pessoais
15252,386314,Enxaguante Bucal Antisséptico Zero Álcool Ment...,1127672,"""O que é e para que serve? Você sabia que os d...",https://static.paodeacucar.com/img/uploads/1/9...,35.99,Listerine,22,9,600.0,5,cuidados-pessoais
15253,591435,Australian Gold Protetor Solar Corporal FPS 50...,1311546,Australian Gold Protetor Solar Corporal FPS 50...,https://static.paodeacucar.com/img/uploads/1/5...,75.90,Australian Gold,20,4.6,227.0,7.8,cuidados-pessoais


In [30]:
PATH_TO_SAVE = '..\data\produtos2.csv'

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Arklok\AppData\Local\Temp\ipykernel_3404\466827048.py:1: SyntaxWarning: invalid escape sequence '\d'
  PATH_TO_SAVE = '..\data\produtos2.csv'


In [31]:
old_df = pd.read_csv(PATH_TO_SAVE.replace('2', ''), sep='|', encoding='utf-8')

In [32]:
df_purenew = df[~df['id'].isin(old_df['id'])]

In [33]:
exitc = pd.concat([old_df, df_purenew], ignore_index=True, sort=False, axis='index')

In [34]:
def categorizar(row):
    if any(str(row['nome']).lower().startswith(c) for c in [
                'óleo',
                'sal', 
                'leite', 
                'arroz', 
                'feijão', 
                'fubá', 
                'farinha', 
                'azeite', 
                'café', 
                'molho',
                'açúcar',
                'macarrão',
                'vinagre',
            ]
        ) and (
            v not in row['nome'].lower() for v in [
                'parbolizado', 'oriental' 
            ]
        ):
        return 'essencial'
    
    elif any(c in str(row['nome']).lower()for c in [
                'file', 'peito', 'bovino', 'paleta', 'frango', 'lombo', 'suíno', 'linguíça', 'patinho', 'maminha', 'coxa', 'mole'
            ]
        ):
        return 'açougue'

    else:
        return row['categoria']

In [35]:
exitc['categoria'] = exitc.apply(categorizar, axis=1)

In [36]:
exitc.to_csv(PATH_TO_SAVE, sep='|', encoding='utf-8')